# Circuit learning module: Lambeq manually with SPSA and JAX

This module performs the optimization of the parametrized circuit manually compared to Lambeq's automatic QuantumTrainer class. I created this because I wanted to have more control over the optimization process and debug it better. The code is based on the workflow presented in https://github.com/CQCL/Quanthoven.

In [1]:
import warnings
import json
import os
import sys
import glob
from math import ceil
from pathlib import Path
from jax import numpy as np
from sympy import default_sort_key
import numpy
import pickle
import matplotlib.pyplot as plt

import jax
from jax import jit
from noisyopt import minimizeSPSA, minimizeCompass

from discopy.quantum import Circuit
from discopy.tensor import Tensor
from discopy.utils import loads
#from pytket.extensions.qiskit import AerBackend
#from pytket.extensions.qulacs import QulacsBackend
#from pytket.extensions.cirq import CirqStateSampleBackend
backend = None

from utils import *
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

warnings.filterwarnings('ignore')
this_folder = os.path.abspath(os.getcwd())
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ["JAX_PLATFORMS"] = "cpu"

SEED = 0

# This avoids TracerArrayConversionError from jax
Tensor.np = np

rng = numpy.random.default_rng(SEED)
numpy.random.seed(SEED)

## Read circuit data

We read the circuits from the pickled files. Select if we perform binary classification or multi-class classification. Give number of qubits to create classes:
- 1 qubits -> 2^1 = 2 classes i.e. binary classification
- 2 qubits -> 2^2 = 4 classes
- ...
- 5 qubits -> 2^5 = 32 classes, etc.

In [2]:
# Select workload
workload = "execution_time"
#workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

loss = multi_class_loss
acc = multi_class_acc

if classification == 1:
    loss = bin_class_loss
    acc = bin_class_acc

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//" + workload + "//" + workload_size + "//circuits//" + str(classification) + "//" + str(layers) + "_layer//" + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count) + "_n_wire_count//"

training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [3]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [4]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

## Lambeq optimizer

## Model

In [5]:
def make_pred_fn(circuits):
    # In the case we want to use other backends. 
    # Currently does not work properly.
    if backend:
        compiled_circuits1 = backend.get_compiled_circuits([c.to_tk() for c in circuits])
        circuits = [Circuit.from_tk(c) for c in compiled_circuits1]
        
    circuit_fns = [c.lambdify(*parameters) for c in circuits]
    
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

## Loss function and evaluation

In [6]:
def make_cost_fn(pred_fn, labels):
    def cost_fn(params, **kwargs):
        predictions = pred_fn(params)

        cost = loss(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)

        return cost

    costs, accuracies = [], []
    return cost_fn, costs, accuracies

## Minimization with noisyopt

In [7]:
def initialize_parameters(old_params, old_values, new_params):
    new_values = list(numpy.array(rng.random(len(new_params))))
    old_param_dict = {}
    for p, v in zip(old_params, old_values):
        old_param_dict[p] = v
        
    parameters = sorted(set(old_params + new_params), key=default_sort_key)
    values = []
    for p in parameters:
        if p in old_param_dict:
            values.append(old_param_dict[p])
        else:
            values.append(new_values.pop())
            
    return parameters, np.array(values)

In [8]:
EPOCHS = 4000
initial_number_of_circuits = 104
syms = {}
limit = False
all_training_keys = list(training_circuits.keys())
initial_circuit_keys = all_training_keys[:initial_number_of_circuits + 1]
current_training_circuits = {}
result_file = workload + "_" + workload_size + "_noisyopt_" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params)

for k in initial_circuit_keys:
    current_training_circuits[k] = training_circuits[k]
    
syms = get_symbols(current_training_circuits)
parameters = sorted(syms, key=default_sort_key)
if initial_number_of_circuits > 5 and os.path.exists("points//" + result_file + ".npz"):
    with open("points//" + result_file + ".npz", "rb") as f:
        print("Loading parameters from file " + result_file)
        npzfile = np.load(f)
        init_params_spsa = npzfile['arr_0']
else:
    print("Initializing new parameters")
    init_params_spsa = np.array(rng.random(len(parameters)))
result = None
run = 0

Loading parameters from file execution_time_main_noisyopt_2_1_3


In [ ]:
for i, key in enumerate(all_training_keys[initial_number_of_circuits:]):
    print("Progress: ", round((i + initial_number_of_circuits)/len(all_training_keys), 3))
    
    if len(syms) == len(get_symbols(current_training_circuits)) and i > 0:
        if i != len(all_training_keys[1:]):
            current_training_circuits[key] = training_circuits[key]
            new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
            if result:
                parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)
                #continue
            else:
                syms = get_symbols(current_training_circuits)
                parameters = sorted(syms, key=default_sort_key)
                init_params_spsa = np.array(rng.random(len(parameters)))
    else:
        run += 1
    
    # Select those circuits from test and validation circuits which share the parameters with the current training circuits
    current_validation_circuits = select_circuits(current_training_circuits, validation_circuits)
    current_test_circuits = select_circuits(current_training_circuits, test_circuits)
    
    if len(current_validation_circuits) == 0 or len(current_test_circuits) == 0:
        continue
    
    # Create lists with circuits and their corresponding label
    training_circuits_l, training_data_labels_l = construct_data_and_labels(current_training_circuits, training_data_labels)
    validation_circuits_l, validation_data_labels_l = construct_data_and_labels(current_validation_circuits, validation_data_labels)
    test_circuits_l, test_data_labels_l = construct_data_and_labels(current_test_circuits, test_data_labels)
    
    # Limit the number of validation and test circuits to 20% of number of the training circuits
    if limit:
        val_test_circ_size = ceil(len(current_training_circuits))
        if len(current_validation_circuits) > val_test_circ_size:
            validation_circuits_l = validation_circuits_l[:val_test_circ_size]
            validation_data_labels_l = validation_data_labels_l[:val_test_circ_size]
        if len(current_test_circuits) > val_test_circ_size:
            test_circuits_l = test_circuits_l[:val_test_circ_size]
            test_data_labels_l = test_data_labels_l[:val_test_circ_size]
    
    stats = f"Number of training circuits: {len(training_circuits_l)}   "\
        + f"Number of validation circuits: {len(validation_circuits_l)}   "\
        + f"Number of test circuits: {len(test_circuits_l)}   "\
        + f"Number of parameters in model: {len(set([sym for circuit in training_circuits_l for sym in circuit.free_symbols]))}"
    
    with open("results//" + result_file + ".txt", "a") as f:
        f.write(stats + "\n")
    
    print(stats)
    
    train_pred_fn = jit(make_pred_fn(training_circuits_l))
    dev_pred_fn = jit(make_pred_fn(validation_circuits_l))
    test_pred_fn = make_pred_fn(test_circuits_l)
    
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)
    
    def callback_fn(xk):
        #print(xk)
        valid_loss = dev_cost_fn(xk)
        train_loss = numpy.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
        train_acc = numpy.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
        valid_acc = numpy.around(float(dev_accs[-1]), 4)
        iters = int(len(train_accs)/2)
        if iters % 200 == 0:
            info = f"Epoch: {iters}   "\
            + f"train/loss: {train_loss}   "\
            + f"valid/loss: {numpy.around(float(valid_loss), 4)}   "\
            + f"train/acc: {train_acc}   "\
            + f"valid/acc: {valid_acc}"
        
            with open("results//" + result_file + ".txt", "a") as f:
                f.write(info + "\n")
                
            print(info, file=sys.stderr)
        return valid_loss
    
    a_value = 0.0053
    c_value = 0.0185
            
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)

    result = minimizeSPSA(train_cost_fn, x0=init_params_spsa, a = a_value, c = c_value, niter=EPOCHS, callback=callback_fn)
    #result = minimizeCompass(train_cost_fn, x0=init_params_spsa, redfactor=2.0, deltainit=1.0, deltatol=0.001, feps=1e-15, errorcontrol=True, funcNinit=30, funcmultfactor=2.0, paired=True, alpha=0.05, callback=callback_fn)

    figure_path = this_folder + "//results//" + result_file + ".png"
    visualize_result_noisyopt(result, make_cost_fn, test_pred_fn, test_data_labels_l, train_costs, train_accs, dev_costs, dev_accs, figure_path, result_file)
    
    run += 1
    #EPOCHS += 100
    syms = get_symbols(current_training_circuits)
    
    # Extend for the next optimization round
    current_training_circuits[key] = training_circuits[key]
    new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
    parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)

Progress:  0.232
Number of training circuits: 104   Number of validation circuits: 108   Number of test circuits: 108   Number of parameters in model: 237


Epoch: 200   train/loss: 31.2692   valid/loss: 38.7658   train/acc: 0.4423   valid/acc: 0.2963
Epoch: 400   train/loss: 27.7339   valid/loss: 39.567   train/acc: 0.5288   valid/acc: 0.3056
Epoch: 600   train/loss: 26.7964   valid/loss: 39.9721   train/acc: 0.4712   valid/acc: 0.3519
Epoch: 800   train/loss: 25.5043   valid/loss: 41.6416   train/acc: 0.4712   valid/acc: 0.3333
Epoch: 1000   train/loss: 24.5754   valid/loss: 40.6112   train/acc: 0.5   valid/acc: 0.3426
Epoch: 1200   train/loss: 24.0318   valid/loss: 41.2406   train/acc: 0.5865   valid/acc: 0.3519
Epoch: 1400   train/loss: 23.5094   valid/loss: 41.972   train/acc: 0.625   valid/acc: 0.3148
Epoch: 1600   train/loss: 22.9935   valid/loss: 42.7551   train/acc: 0.6346   valid/acc: 0.3056
Epoch: 1800   train/loss: 23.0835   valid/loss: 40.6754   train/acc: 0.6058   valid/acc: 0.3241
Epoch: 2000   train/loss: 22.1655   valid/loss: 41.9238   train/acc: 0.6346   valid/acc: 0.3241
Epoch: 2200   train/loss: 22.0762   valid/loss: 43

Test accuracy: 0.3611111111111111
Progress:  0.234
Number of training circuits: 105   Number of validation circuits: 108   Number of test circuits: 108   Number of parameters in model: 237


Epoch: 200   train/loss: 30.4317   valid/loss: 39.2071   train/acc: 0.4286   valid/acc: 0.2593
Epoch: 400   train/loss: 26.6811   valid/loss: 41.4961   train/acc: 0.5238   valid/acc: 0.287
Epoch: 600   train/loss: 25.8112   valid/loss: 43.8308   train/acc: 0.5905   valid/acc: 0.3333
Epoch: 800   train/loss: 25.0482   valid/loss: 45.0068   train/acc: 0.5905   valid/acc: 0.3056
Epoch: 1000   train/loss: 23.7639   valid/loss: 42.6297   train/acc: 0.6095   valid/acc: 0.3148
Epoch: 1200   train/loss: 22.9263   valid/loss: 44.8098   train/acc: 0.6476   valid/acc: 0.3426
Epoch: 1400   train/loss: 23.0443   valid/loss: 45.0096   train/acc: 0.6286   valid/acc: 0.3333
Epoch: 1600   train/loss: 22.4966   valid/loss: 44.0659   train/acc: 0.6571   valid/acc: 0.3148
Epoch: 1800   train/loss: 22.1181   valid/loss: 44.8796   train/acc: 0.6571   valid/acc: 0.3333
Epoch: 2000   train/loss: 22.0123   valid/loss: 45.1859   train/acc: 0.6571   valid/acc: 0.3333
Epoch: 2200   train/loss: 21.9127   valid/los

Test accuracy: 0.3148148148148148
Progress:  0.237
Number of training circuits: 106   Number of validation circuits: 108   Number of test circuits: 108   Number of parameters in model: 237


...

Epoch: 200   train/loss: 46.185   valid/loss: 40.0856   train/acc: 0.3043   valid/acc: 0.2252
Epoch: 400   train/loss: 43.242   valid/loss: 38.5713   train/acc: 0.3913   valid/acc: 0.3514
Epoch: 600   train/loss: 41.5414   valid/loss: 40.0786   train/acc: 0.471   valid/acc: 0.3243
Epoch: 800   train/loss: 38.3699   valid/loss: 40.2618   train/acc: 0.4565   valid/acc: 0.3514
Epoch: 1000   train/loss: 36.3013   valid/loss: 42.6399   train/acc: 0.5507   valid/acc: 0.3243
Epoch: 1200   train/loss: 35.8547   valid/loss: 43.5478   train/acc: 0.5217   valid/acc: 0.3243
Epoch: 1400   train/loss: 34.1183   valid/loss: 42.8883   train/acc: 0.529   valid/acc: 0.3694
Epoch: 1600   train/loss: 33.8337   valid/loss: 43.0534   train/acc: 0.5725   valid/acc: 0.3153
Epoch: 1800   train/loss: 33.9603   valid/loss: 42.7621   train/acc: 0.5797   valid/acc: 0.3243
Epoch: 2000   train/loss: 33.2565   valid/loss: 46.7545   train/acc: 0.5942   valid/acc: 0.3153
Epoch: 2200   train/loss: 33.0541   valid/loss: 

Test accuracy: 0.37272727272727274
Progress:  0.31
Number of training circuits: 139   Number of validation circuits: 111   Number of test circuits: 110   Number of parameters in model: 264


Epoch: 200   train/loss: 45.7834   valid/loss: 40.846   train/acc: 0.4029   valid/acc: 0.2523
Epoch: 400   train/loss: 39.2359   valid/loss: 39.6451   train/acc: 0.446   valid/acc: 0.3243
Epoch: 600   train/loss: 39.3643   valid/loss: 41.2179   train/acc: 0.4101   valid/acc: 0.3063
Epoch: 800   train/loss: 37.753   valid/loss: 43.7796   train/acc: 0.4892   valid/acc: 0.2973
Epoch: 1000   train/loss: 34.3916   valid/loss: 41.5832   train/acc: 0.518   valid/acc: 0.3423
Epoch: 1200   train/loss: 33.3193   valid/loss: 41.6304   train/acc: 0.5971   valid/acc: 0.3243
Epoch: 1400   train/loss: 32.8105   valid/loss: 42.0553   train/acc: 0.6115   valid/acc: 0.3153
Epoch: 1600   train/loss: 32.2223   valid/loss: 41.4708   train/acc: 0.5899   valid/acc: 0.3243
Epoch: 1800   train/loss: 31.8784   valid/loss: 43.1364   train/acc: 0.5899   valid/acc: 0.3333
Epoch: 2000   train/loss: 31.4924   valid/loss: 44.1607   train/acc: 0.6331   valid/acc: 0.3333
Epoch: 2200   train/loss: 30.8659   valid/loss: 

Test accuracy: 0.4090909090909091
Progress:  0.312
Number of training circuits: 140   Number of validation circuits: 111   Number of test circuits: 110   Number of parameters in model: 264


Epoch: 200   train/loss: 45.9172   valid/loss: 43.2758   train/acc: 0.3857   valid/acc: 0.2523
Epoch: 400   train/loss: 40.2425   valid/loss: 47.0084   train/acc: 0.4571   valid/acc: 0.2432
Epoch: 600   train/loss: 38.3913   valid/loss: 48.2559   train/acc: 0.4786   valid/acc: 0.2613
Epoch: 800   train/loss: 36.9181   valid/loss: 47.7688   train/acc: 0.5357   valid/acc: 0.2342
Epoch: 1000   train/loss: 35.9369   valid/loss: 45.3334   train/acc: 0.5357   valid/acc: 0.2883
Epoch: 1200   train/loss: 34.4586   valid/loss: 42.1376   train/acc: 0.5714   valid/acc: 0.3063
Epoch: 1400   train/loss: 33.4822   valid/loss: 44.2543   train/acc: 0.5714   valid/acc: 0.3153
Epoch: 1600   train/loss: 33.3898   valid/loss: 43.6721   train/acc: 0.5571   valid/acc: 0.2883
Epoch: 1800   train/loss: 32.5872   valid/loss: 42.3919   train/acc: 0.5857   valid/acc: 0.3153
Epoch: 2000   train/loss: 32.6071   valid/loss: 42.8569   train/acc: 0.6071   valid/acc: 0.3153
